Extracts relevant variables from Dealscan

In [ ]:
import joblib
import pandas as pd
import warnings
import re
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)
warnings.filterwarnings('ignore')
tqdm.pandas()

In [ ]:
# read new dataset 

dat_df = joblib.load('../../rawdata/dealscan/dealscan_new.pkl')

dat_df.set_index(['lpc_tranche_id','tranche_active_date'], inplace=True)
dat_df.sort_index(inplace=True)
dat_df.reset_index(inplace=True)



In [ ]:
for i in dat_df.columns:
    print(i)

In [ ]:
# keep relevant variables

dat_df_1 = dat_df[['borrower_id','lpc_deal_id','lpc_tranche_id','deal_active_date',
                    'deal_currency','deal_amended','deal_amount','tranche_amount',
                    'tranche_active_date','tranche_maturity_date','market_segment','tranche_type',
                    'covenants', 'all_covenants_financial','deal_active','deal_refinancing',
                    'base_reference_rate','all_in_spread_drawn_bps','all_in_spread_undrawn_bps'
                  ]]
dat_df_1['covenants'] = dat_df_1['covenants'].map({'Yes':1,'No':0})

In [ ]:
# same tranche gets repeated for different lenders, so just keep unique 

dat_df_1 = dat_df_1\
    .groupby(['lpc_tranche_id','tranche_active_date']).first()
dat_df_1.reset_index(inplace=True)


In [ ]:
pd.set_option('max_rows',20)
pd.set_option('max_colwidth',None)
dat_df_1.head(20)

In [ ]:
def read_covenants(str_in):
    
    out = dict()
    
    str_in = np.nan_to_num(str_in, nan=-99)
    
    if str_in and str_in != -99:
        
        variable = re.findall(r'[A-z\s.]*.(?=:)', str_in)
        value = re.findall(r'(?:(?<=to )|(?<=: )|(?<=is ))[-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?', str_in)
        
        out = dict([(i[0].strip(),float(i[1])) for i in zip(variable, value)])
        
    return out
    

In [ ]:
def read_marketsegment(str_in):
    
    out = dict()
    
    str_in = np.nan_to_num(str_in, nan=-99)
    
    if str_in and str_in != -99:
        
        regex = re.compile('leverage',re.IGNORECASE)
        leverage = int(len(re.findall(regex,str_in))>0)
        
        regex = re.compile('covenant lite', re.IGNORECASE)
        covlite = int(len(re.findall(regex,str_in))>0)
        
        out = {'levloan': leverage, 'covlite': covlite}
        
    return out
        
    

In [ ]:
def read_tranchetype(str_in):
    
    revolver = 0
    
    str_in = np.nan_to_num(str_in, nan=-99)
    
    if str_in and str_in != -99:
        
        regex = r'\b(?:Revolver|364-Day Facility)'
        revolver = int(len(re.findall(regex,str_in))>0)
        
    return revolver

In [ ]:
# extract covenants from covenant_financials

dictList = dat_df_1['all_covenants_financial'].progress_apply(read_covenants)
dictList1 = pd.DataFrame.from_dict(list(dictList))

dat_df_1 = dat_df_1.join(dictList1)

In [ ]:
mktseg = dat_df_1['market_segment'].progress_apply(read_marketsegment)
mktseg = pd.DataFrame.from_dict(list(mktseg))
dat_df_1 = dat_df_1.join(mktseg)

In [ ]:
dat_df_1['revolver'] = dat_df_1['tranche_type'].progress_apply(read_tranchetype)
dat_df_1.loc[dat_df_1.revolver==1,'revolver_amount'] = dat_df_1.loc[dat_df_1.revolver==1,'tranche_amount']

In [ ]:
# check if extraction procedure is successful
# if successful, this should yield no rows
# note: there are fields where covenants == 0 but all_covenants_financial is not empty

tmp = dat_df_1.copy()
tmp['parseflag'] = (tmp[dictList1.columns.values.tolist()].isna().sum(axis=1)!=len(dictList1.columns))
tmp['parseflag'] = tmp['parseflag'].astype(int)

# covenant indicator = 1 but text not successfully parsed
tmp[(tmp.covenants==1)&(tmp.parseflag==0)&(tmp.all_covenants_financial.notnull())]

In [ ]:
# get borrower id link 
id_df = joblib.load('../../rawdata/dealscan/lpc_loanconnector_company_id_map.pkl')
id_df.rename({'loanconnector_company_id':'borrower_id'}, axis=1, inplace=True)
id_df.head()

In [ ]:
# get chava roberts link table
roberts_df = pd.read_excel("../../rawdata/dealscan/Dealscan-Compustat_Linking_Database.xlsx", sheet_name='link_data')


In [ ]:
# keep one identifer per borrower-facility start date
roberts_df['year'] = roberts_df.facstartdate.dt.year
roberts_df_1 = roberts_df[['bcoid','gvkey','facstartdate','fic']].drop_duplicates(subset=['bcoid','facstartdate'])
roberts_df_1.rename({'bcoid': 'lpc_company_id','facstartdate':'tranche_active_date'}, axis=1, inplace=True)
roberts_df_1.head()

In [ ]:
## do the merging here

dat_df_2 = dat_df_1.copy()

# adjust data type
dat_df_2['tranche_active_date'] = pd.to_datetime(dat_df_2['tranche_active_date'], errors='coerce')
dat_df_2['tranche_maturity_date'] = pd.to_datetime(dat_df_2['tranche_maturity_date'], errors='coerce')
dat_df_2['deal_active_date'] = pd.to_datetime(dat_df_2['deal_active_date'], errors='coerce')

# merge with old company identifiers
dat_df_2 = dat_df_2.merge(id_df, on='borrower_id', how='outer', indicator=True, validate='many_to_one')
print(dat_df_2._merge.value_counts())
dat_df_2 = dat_df_2[dat_df_2._merge=='both'].reset_index(drop=True)
dat_df_2.drop(['_merge'],axis=1,inplace=True)


In [ ]:
# merge with chava-roberts link table

dat_df_2['lpc_company_id'] = dat_df_2['lpc_company_id'].astype(int)
dat_df_2 = dat_df_2.merge(roberts_df_1, on=['lpc_company_id','tranche_active_date'], how='left', indicator=True, validate='many_to_one')
print(dat_df_2._merge.value_counts())
print(dat_df_2.gvkey.isna().sum())

In [ ]:
# carry forward gvkey values for those with previous match (assume no change)

dat_df_2.set_index(['lpc_company_id','tranche_active_date'], inplace=True)
dat_df_2.sort_index(inplace=True)
dat_df_2['gvkey'] = dat_df_2['gvkey'].groupby(level=0).fillna(method='ffill')

# carry backward gvkey values for those with future match 
dat_df_2['gvkey'] = dat_df_2['gvkey'].groupby(level=0).fillna(method='bfill')

# formatting
dat_df_2.reset_index(inplace=True)
dat_df_2.drop(['_merge'],axis=1,inplace=True)

dat_df_2.gvkey.isna().sum()

In [ ]:
dat_df_2.head(10)

In [ ]:
# save tranche level data to compute average spreads 

dat_df_2.drop(['all_covenants_financial','market_segment'],axis=1, inplace=True)
dat_df_2.to_stata('../../rawdata/dealscan/dealscan_new_2.dta')

### Collapse to package level

In [ ]:
# collapse to package level (covenants apply to all loans in package)

dat_df_3 = dat_df_2.copy()

# drop variables that are not meaningful at the deal level
dat_df_3.drop(['tranche_amount','market_segment'], axis=1, inplace=True)

# for each deal_id-tranche_active_date, covenant applies to earliest tranche and latest maturity date
# see Chava and Roberts 2008 p.2092 (tranche_active_date keeps track of amendments to tranches)
dat_df_3['max_deal_maturity_date'] = dat_df_3.groupby(['lpc_deal_id','tranche_active_date'])['tranche_maturity_date'].transform('max')
dat_df_3['min_deal_active_date'] = dat_df_3.groupby(['lpc_deal_id','tranche_active_date'])['tranche_active_date'].transform('min')
dat_df_3['revolver_deal'] = dat_df_3.groupby(['lpc_deal_id','tranche_active_date'])['revolver'].transform('max')
dat_df_3['revolver_amount_deal'] = dat_df_3.groupby(['lpc_deal_id','tranche_active_date'])['revolver_amount'].transform('max')
dat_df_3['levloan_deal'] = dat_df_3.groupby(['lpc_deal_id','tranche_active_date'])['levloan'].transform('max')
dat_df_3['covlite_deal'] = dat_df_3.groupby(['lpc_deal_id','tranche_active_date'])['covlite'].transform('max')
dat_df_3.drop(['revolver','revolver_amount','levloan','covlite'], axis=1, inplace=True)

dat_df_3 = dat_df_3\
    .groupby(['lpc_deal_id','tranche_active_date']).first()

dat_df_3.reset_index(inplace=True)

# share of matches after carry forward and carry back
# 51 percent of loans with matched gvkey
dat_df_3.gvkey.notnull().sum() / dat_df_3.shape[0]

In [ ]:
# replace maturity date by amended loan's active date if loan is amended 
# check if new active date is before existing maturity date
dat_df_3.set_index(['lpc_deal_id','tranche_active_date'], inplace=True)
dat_df_3.sort_index(inplace=True)
dat_df_3['adj_deal_maturity_date'] = dat_df_3.groupby(level=0)['min_deal_active_date'].shift(-1)

# if no amendment, then just use existing deal maturity date
mask = dat_df_3.adj_deal_maturity_date.isnull()
dat_df_3.loc[mask,'adj_deal_maturity_date'] = dat_df_3.loc[mask,'max_deal_maturity_date']

# check if amendment start date after existing deal maturity date. if yes, then use existing maturity date
mask = dat_df_3.adj_deal_maturity_date > dat_df_3.max_deal_maturity_date
dat_df_3.loc[mask,'adj_deal_maturity_date'] = dat_df_3.loc[mask,'max_deal_maturity_date']

dat_df_3.reset_index(inplace=True)

In [ ]:
# check if there are unmatched values 
#   92 facilities in CR table with unmatched values
#   30 facilities for US incorp firms in Chava-Roberts table unmatched (15 firms)

list_ = dat_df_3[dat_df_3.gvkey.isna()==True].lpc_company_id.unique()

aux = roberts_df[roberts_df.bcoid.isin(list_)]

aux[aux.fic=="USA"].company.value_counts()

In [ ]:
dat_df_3[(dat_df_3.tranche_active_date.dt.year>=2010)&(dat_df_3.covenants==1)].head(50)

## convert to long format 

In [ ]:
# convert data to long format (for merging in stata)

dat_df_4 = dat_df_3.copy()

# cov list
cov_list = ['Tangible Net Worth',
            'Min. Interest Coverage Ratio', 
            'Min. Current Ratio',
            'Max. Debt to Cash Flow', 
            'Net Worth',
            'Max. Debt to Tangible Net Worth Ratio',
            'Min. Debt Service Coverage Ratio', 
            'Min. Fixed Charge Coverage Ratio',
            'Max. Sr. Debt to Cash Flow', 
            'Max. Leverage Ratio',
            'Min. Cash Interest Coverage Ratio', 
            'Max. Debt to Equity Ratio',
            'Max. Loan to Value Ratio']

dat_df_4 = pd.melt(dat_df_4, id_vars=['lpc_deal_id','tranche_active_date'], value_vars=cov_list) \
            .sort_values(['lpc_deal_id','tranche_active_date'])

# merge in remaining values
tmp = dat_df_3[['lpc_company_id', 
                'tranche_active_date', 
                'borrower_id',
                'lpc_deal_id', 
                'min_deal_active_date', 
                'max_deal_maturity_date',
                'adj_deal_maturity_date',
                'deal_currency',
                'deal_active',
                'deal_amended', 
                'deal_amount', 
                'covenants',
                'covlite_deal',
                'levloan_deal',
                'revolver_deal',
                'revolver_amount_deal',
                'gvkey']]
dat_df_4 = dat_df_4.merge(tmp, on=['lpc_deal_id','tranche_active_date'], how='left', validate='many_to_one')

In [ ]:
# rename categories

dict_ = {'Max. Debt to Cash Flow': 'Max. Debt to EBITDA',
         'Max. Sr. Debt to Cash Flow': 'Max. Senior Debt to EBITDA',
         'Min. Interest Coverage Ratio': 'Min. Interest Coverage',
         'Min. Fixed Charge Coverage Ratio': 'Min. Fixed Charge Coverage',
         'Min. Debt Service Coverage Ratio': 'Min. Debt Service Coverage',
         'Min. Cash Interest Coverage Ratio': 'Min. Cash Interest Coverage',
         'Max. Debt to Equity Ratio': 'Max. Debt to Equity',
         'Max. Leverage Ratio': 'Max. Leverage ratio',
         'Tangible Net Worth': 'Tangible Net Worth',
         'Net Worth': 'Net Worth',
         'Min. Current Ratio': 'Min. Current Ratio',
         'Max. Loan to Value Ratio': 'Max. Loan to Value ratio',
         'Max. Debt to Tangible Net Worth Ratio': 'Max. Debt to Tangible Net Worth'
         }

dat_df_4['variable'] = dat_df_4['variable'].map(dict_)
dat_df_4.rename({'variable':'covenant','covenants':'has_covenant','value': 'covthreshold'}, axis=1, inplace=True)


In [ ]:
dat_df_4.to_stata('../../data/dealscan_combined_long_new_2.dta')

## some analysis on raw data

In [ ]:
# unique identifiers: lpc_deal_id, tranche_active_date

dat_df_2[dat_df_2.deal_amended=='Yes'].head(20)

In [ ]:
# number of new issues over time in Dealscan

tmp = dat_df_2.copy()
tmp['year'] = tmp.tranche_active_date.dt.year
tmp = tmp[tmp.year.between(1994,2021)]
tmp = tmp[tmp.fic=='USA']

# tmp1 = tmp.drop_duplicates(subset=['lpc_deal_id'], keep='first')
tmp.year.value_counts().sort_index().plot(label='new loans')

tmp2 = tmp.drop_duplicates(subset='lpc_company_id', keep='first')
tmp2.year.value_counts().sort_index().plot(label='new companies')
plt.legend()


In [ ]:
# compute average duration of loan contracts 

tmp['duration'] = tmp.tranche_maturity_date.dt.year - tmp.tranche_active_date.dt.year
plot_ = tmp.groupby('year')['duration'].quantile([0.05,0.5,0.95]).loc[1994:]
plot_ = pd.DataFrame(plot_).reset_index()
plot_.columns=['year','quantiles','value']
plot_ = plot_.pivot(index='year',columns='quantiles')
plot_.columns = ['p5', 'p50','p95']
plt.plot(plot_.index,plot_['p5'], label='p5')
plt.plot(plot_.index,plot_['p50'], label='p50')
plt.plot(plot_.index,plot_['p95'], label='p95')

plt.legend()


In [ ]:
dat_df_3 = dat_df_1.copy()

# adjust data type
dat_df_3['tranche_active_date'] = pd.to_datetime(dat_df_3['tranche_active_date'], errors='coerce')
dat_df_3['tranche_maturity_date'] = pd.to_datetime(dat_df_3['tranche_maturity_date'], errors='coerce')
dat_df_3['deal_active_date'] = pd.to_datetime(dat_df_3['deal_active_date'], errors='coerce')

dat_df_3['duration'] = dat_df_3.tranche_maturity_date.dt.year - dat_df_3.tranche_active_date.dt.year
dat_df_3['year'] = dat_df_3.tranche_active_date.dt.year

dat_df_3 = dat_df_3[dat_df_3.year.between(1994,2021)]
# dat_df_3 = dat_df_3[dat_df_3.fic=='USA']

plot_ = dat_df_3.groupby('year')['duration'].mean().loc[1994:]
plot_ = pd.DataFrame(plot_).reset_index()
plt.scatter(plot_['year'],plot_['duration'])
plt.plot(plot_['year'],plot_['duration'])
plt.xlabel('Year of origination')
plt.ylabel('Average deal maturity (years)')

In [ ]:
plot_

In [ ]:
tmp = dat_df_2.copy()
tmp = tmp[tmp.fic=='USA']

tmp = tmp.drop_duplicates(subset='lpc_deal_id', keep='first')
tmp.groupby('year')['deal_amount'].sum()

## scratch

In [ ]:
# get list of covenants

# ebc_list = ['max_debt_to_cash_flow', 
#             'max_sr_debt_to_cash_flow', 
#             'min_interest_coverage_ratio', 
#             'min_fixed_charge_coverage_ratio',
#             'min_debt_service_coverage_ratio',
#             'min_cash_interest_coverage_ratio']
# abc_list = ['max_debt_to_equity_ratio',
#             'max_leverage_ratio',
#             'min_current_ratio',
#             'max_loan_to_value_ratio',
#             'max_debt_to_tangible_net_worth']

# cov_list = ebc_list + abc_list

In [ ]:
# carryforward ratios for same trench as long as covenant == Yes (ratios not repeated if amended )

# dat_df_1.sort_index(inplace=True)

# for i in tqdm(cov_list):
#     dat_df_1[i] = dat_df_1.loc[dat_df_1['covenants']=='Yes',i].groupby(level=0).fillna(method='ffill')

# dat_df_1.reset_index(inplace=True)

In [ ]:
# generate variable for number of ebcs and abcs (asset based)
# note: if tranche is amended, new entry does not record covenant unless amended, but covenants record "Yes"
# see entry numbers 647:648 for comparison

# dat_df_1['num_ebc'] = len(ebc_list) - dat_df_1[ebc_list].isnull().sum(axis=1)
# dat_df_1['num_abc'] = len(abc_list) - dat_df_1[abc_list].isnull().sum(axis=1)
# dat_df_1['covenants'] = dat_df_1.covenants.map({'Yes':1,'No':0})

In [ ]:
# in this example, no covenants are recorded, but there is a non null entry in "all_covenants_financial". See lpc_tranche_id==100436
# there are also cases where covenant = 1 but no entries in covenants or 'all_covenants_financial'. See lpc_tranche_id==100305

# pd.set_option('max_columns', None)
# dat_df_1[dat_df_1.lpc_tranche_id=='100436'].reset_index(drop=True)['all_covenants_financial'][1]

In [ ]:
# check for misclassified cases (generalizing the case above)
# 2 percent of cases are like this

# dat_df_1[((dat_df_1.num_ebc==0)&(dat_df_1.num_abc==0))&(dat_df_1.covenants==1)].shape[0] / dat_df_1.shape[0]

In [ ]:
# function to extract covenant ratio from string 
# data encodes it as a string

# def read_string(str_in):
    
#     out = None
    
#     str_in = np.nan_to_num(str_in, nan=-99)
    
#     if str_in and str_in != -99:
#         aux = re.search(r'((?<=to ).*(?=:1))|^[.*(?=:1$)]', str_in)
#         if aux:
#             out = float(aux.group())
    
#     return out

In [ ]:
# extract covenant ratios

# cov_list = ['max_debt_to_cash_flow', 
#             'max_sr_debt_to_cash_flow', 
#             'min_interest_coverage_ratio', 
#             'min_fixed_charge_coverage_ratio',
#             'min_debt_service_coverage_ratio',
#             'min_cash_interest_coverage_ratio',
#             'max_debt_to_equity_ratio',
#             'max_leverage_ratio',
#             'min_current_ratio',
#             'max_loan_to_value_ratio',
#             'max_debt_to_tangible_net_worth']

# for i in tqdm(cov_list):
    
#     aux = dat_df_1[i].apply(read_string)
#     dat_df_1[i] = aux


In [ ]:
# def custom_round(x, base=5):
#     return base * round(float(x)/base)

In [ ]:
# # second round of merging, use closest match in 5 years 

# dat_df_3 = dat_df_2.copy()
# dat_df_3['year10'] = dat_df_2['deal_active_date'].apply(lambda x: custom_round(x.year, base=10))
# dat_df_3.head()

In [ ]:
# roberts_df_2 = roberts_df_1.copy()

# roberts_df_2.dropna(subset=['year'],inplace=True)
# roberts_df_2['year10'] = roberts_df_2['year'].apply(lambda x: custom_round(x, base=10))
# roberts_df_2.rename({'gvkey':'gvkey2'}, axis=1, inplace=True)

# # if duplicated just keep latest 
# roberts_df_2.set_index(['lpc_company_id','year'], inplace=True)
# roberts_df_2.sort_index(inplace=True)
# roberts_df_2.reset_index(inplace=True)
# roberts_df_2 = roberts_df_2.drop_duplicates(subset=['lpc_company_id','year10'])
# roberts_df_2.drop(['year'],axis=1,inplace=True)
# roberts_df_2.head(20)

In [ ]:
# dat_df_3 = dat_df_3.merge(roberts_df_2, on=['lpc_company_id','year10'], validate='many_to_one', how='left')


In [ ]:
# list_ = dat_df_3[dat_df_3.gvkey2.isna()==True].lpc_company_id.unique()

# aux = roberts_df_2[roberts_df_2.lpc_company_id.isin(list_)]

# aux.shape[0] / roberts_df_2.shape[0]


In [ ]:
# # use first round matching (match exact year), if missing then use second round matching

# dat_df_3['gvkey'] = dat_df_3['gvkey'].fillna(dat_df_3['gvkey2'])

# dat_df_3.drop(['year10','gvkey2'], axis=1, inplace=True)

# dat_df_3.head()


In [ ]:
# list_ = dat_df_3[dat_df_3.gvkey.isna()==True].lpc_company_id.unique()

# aux = roberts_df_2[roberts_df_2.lpc_company_id.isin(list_)]

# aux.shape[0]

In [ ]:
# dat_df_3.gvkey.notnull().sum() / dat_df_3.shape[0]

In [ ]:
# dat_df_3['tranche_maturity_date'] = pd.to_datetime(dat_df_3['tranche_maturity_date'])
# dat_df_3['deal_active_date'] = pd.to_datetime(dat_df_3['deal_active_date'])
# dat_df_3.to_stata('../../data/dealscan_combined_long_new.dta')